# **Basics of GPU Programming**

This tutorial will walk you through the process of converting CPU-based Python code to GPU-accelerated code using NUMBA and CUDA.

## Table of Contents
 1. [CUDA and NUMBA Introduction](#introduction)
 2. [Setting Up Your Environment](#setup)
 3. [Basic Concepts](#concepts)
 4. [Pattern 1: Simple Array Operations](#array-ops)
 5. [Pattern 2: Device Functions](#device-functions)
 6. [Pattern 3: Memory Management](#memory)
 7. [Pattern 4: Kernel Launch Patterns](#kernels)
 8. [Best Practices and Common Pitfalls](#best-practices)

### **CUDA and NUMBA Introduction** <a name="introduction"></a>

CUDA (Compute Unified Device Architecture) is NVIDIA's parallel computing platform and programming model designed for their GPUs. While GPUs were originally designed for rendering graphics, CUDA allows them to be used for general-purpose computing (GPGPU).

##### Key aspects of CUDA:
* Allows direct access to GPU's virtual instruction set
* Enables parallel processing across thousands of GPU cores
* Only works with NVIDIA GPUs
* Traditionally requires writing code in C/C++

##### Why GPUs are good for parallel computing:
* **CPU vs GPU Architecture**:
  * CPUs have few cores (2-64) optimized for sequential tasks
  * GPUs have thousands of smaller cores optimized for parallel tasks

* **Parallel Processing Benefits**:
  * GPUs excel at tasks where the same operation is performed on large amounts of data
  * Common applications:
    * Scientific computing
    * Machine learning
    * Simulation
    * Data processing

### **NUMBA**

Numba is a Just-In-Time (JIT) compiler that translates Python code into optimized machine code. It's particularly valuable because of the following features:

##### 1. Makes GPU Programming Accessible
* **Python-First Approach**: Write CUDA code in Python instead of C/C++
* **Automatic Memory Management**: No manual memory allocation/deallocation needed
* **Simple Decorators**: Use `@cuda.jit` to designate GPU-accelerated functions

##### 2. Offers Multiple Compilation Targets
* **CPU Optimization**: Use `@jit` for faster CPU execution
* **GPU Acceleration**: Use `@cuda.jit` for GPU-based parallel processing
* **Parallel CPU**: Use `@vectorize` for CPU-based parallel execution

##### 3. Key Features
* **NumPy Integration**: 
  * Seamless compatibility with NumPy arrays
  * Direct support for NumPy operations and functions
  
* **Python Subset Support**: 
  * Works with core Python features
  * Supports essential NumPy functionality
  
* **Low-Effort Migration**: 
  * Minimal code changes needed for GPU acceleration
  * Keep existing Python code structure
  
* **Automatic Optimization**: 
  * Smart optimization of numerical computations
  * Efficient loop transformations
  * Vectorization of operations where possible    

## **Setting Up Your Environment** <a name="setup"></a>
 
First, let's import the necessary packages and check if CUDA is available:

In [20]:
from numba import cuda, float64, float32, int32
import numpy as np
import time

# Check if CUDA is available
cuda_available = cuda.is_available()
print(f"CUDA available: {cuda_available}")

if not cuda_available:
    print("\nCUDA is not available on your system. Here's what you can do:\n")
    print("1. Check if you have an NVIDIA GPU:")
    print("   - Windows: Task Manager > Performance tab")
    print("   - Linux: Run 'nvidia-smi' in terminal")
    print("   - Mac: System Report > Hardware > Graphics/Displays\n")
    
    print("2. Install NVIDIA drivers:")
    print("   - Download from: https://www.nvidia.com/Download/index.aspx")
    print("   - Follow installation instructions for your operating system\n")
    
    print("3. Install CUDA Toolkit:")
    print("   - Download from: https://developer.nvidia.com/cuda-downloads")
    print("   - Ensure version compatibility with your Python environment\n")
    
    print("4. Install required Python packages:")
    print("   pip install numba cuda-python\n")
    
    print("5. Common issues:")
    print("   - Mac M1/M2 chips: CUDA is not supported")
    print("   - Virtual Environments: May need to reinstall packages")
    print("   - Linux: May need to set CUDA_HOME environment variable\n")
    
    print("6. Verify installation:")
    print("   - Run 'nvcc --version' in terminal")
    print("   - Check if nvidia-smi shows your GPU\n")
    
    print("Until CUDA is available, you can still follow along with the CPU examples in this tutorial.")
    print("The GPU code patterns will be educational, but you won't be able to execute them.\n")


CUDA available: True


Now we can proceed with the rest of the tutorial, but be aware that if CUDA is not available, 
the GPU-specific code examples will not execute. You can still learn the patterns and concepts, 
which will be useful once you have CUDA properly set up.

If you're using Google Colab as an alternative, you can get access to a GPU by:
1. Go to Runtime > Change runtime type
2. Select GPU from the Hardware accelerator drop-down menu
3. Click Save

## **Basic Concepts** <a name="concepts"></a>

Before diving into code conversion, let's understand some key GPU programming concepts:

- **Thread**: Smallest unit of execution on GPU
- **Block**: Group of threads that can share memory
- **Grid**: Collection of blocks
- **Kernel**: Function that runs on the GPU
- **Device**: The GPU
- **Host**: The CPU

## **Pattern 1: Simple Array Operations** <a name="array-ops"></a>

Let's start with a simple example: squaring each element in an array.
<br>
### CPU Version

In [2]:
def cpu_square_array(arr):
    result = np.zeros_like(arr)
    for i in range(len(arr)):
        result[i] = arr[i] * arr[i]
    return result

# Example usage
data = np.array([1, 2, 3, 4, 5], dtype=np.float64)
result_cpu = cpu_square_array(data)
print("CPU Result:", result_cpu)

CPU Result: [ 1.  4.  9. 16. 25.]


### GPU Version

In [4]:
# The @cuda.jit decorator tells Numba this function will run on the GPU
# This decorator compiles the function into CUDA device code
@cuda.jit
def gpu_square_array(in_arr, out_arr):
    # cuda.grid(1) gives us the absolute thread ID in the 1D grid
    # Each thread will handle one element of the array
    idx = cuda.grid(1)
    
    # We must check if the thread's index is within array bounds
    # This is crucial because we might launch more threads than array elements
    if idx < in_arr.size:
        out_arr[idx] = in_arr[idx] * in_arr[idx]

# Create input data - must specify dtype for GPU compatibility
data = np.array([1, 2, 3, 4, 5], dtype=np.float64)
result_gpu = np.zeros_like(data)

# Calculate grid and block sizes
# threads_per_block: number of threads in each block (hardware limit is typically 1024)
# Choose a multiple of 32 (warp size) for best performance
threads_per_block = 256

# Calculate number of blocks needed:
# - data.size is the total number of elements
# - (data.size + threads_per_block - 1) ensures we round up the division
blocks_per_grid = (data.size + threads_per_block - 1) // threads_per_block

print(f"Grid configuration: {blocks_per_grid} blocks, {threads_per_block} threads per block")

# Transfer data to GPU
# cuda.to_device allocates memory on GPU and copies data from CPU
d_data = cuda.to_device(data)
d_result = cuda.to_device(result_gpu)

# Launch the kernel with our grid configuration
# Syntax: kernel[blocks_per_grid, threads_per_block](arguments...)
gpu_square_array[blocks_per_grid, threads_per_block](d_data, d_result)

# Transfer results back to CPU with copy_to_host()
result_gpu = d_result.copy_to_host()
print("GPU Result:", result_gpu)

Grid configuration: 1 blocks, 256 threads per block
GPU Result: [ 1.  4.  9. 16. 25.]


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Understanding the GPU Implementation

Let's break down the key components and concepts:

1. **Kernel Decoration**
   ```python
   @cuda.jit
   ```
   - This decorator marks the function as a CUDA kernel
   - It tells Numba to compile the function for GPU execution
   - The function will be run in parallel by many GPU threads

2. **Thread Indexing**
   ```python
   idx = cuda.grid(1)
   ```
   - Gets the absolute thread ID in a 1D grid
   - Each thread needs to know which array element it's responsible for
   - The '1' parameter indicates we're working with a 1D grid

3. **Bounds Checking**
   ```python
   if idx < in_arr.size:
   ```
   - Critical for preventing out-of-bounds memory access
   - Necessary because we might launch more threads than array elements
   - GPU threads that exceed array bounds will do nothing

4. **Grid Configuration**
   ```python
   threads_per_block = 256
   blocks_per_grid = (data.size + threads_per_block - 1) // threads_per_block
   ```
   - threads_per_block: how many threads run in parallel within each block
     - Usually a multiple of 32 (warp size)
     - Maximum is typically 1024
   - blocks_per_grid: how many blocks we need
     - Formula ensures we have enough threads to cover all elements
     - The division is rounded up to ensure coverage

5. **Memory Transfers**
   ```python
   d_data = cuda.to_device(data)
   d_result = cuda.to_device(result_gpu)
   result_gpu = d_result.copy_to_host()
   ```
   - to_device(): Allocates GPU memory and copies data from CPU to GPU
   - copy_to_host(): Copies results from GPU back to CPU
   - 'd_' prefix is a common convention for GPU (device) variables

### Performance Comparison

Let's compare the performance of CPU and GPU versions with a larger array:

In [5]:
# Create a larger array for meaningful comparison
large_data = np.random.random(1000000).astype(np.float64)

# Time CPU version
start_cpu = time.perf_counter()
result_cpu = cpu_square_array(large_data)
cpu_time = time.perf_counter() - start_cpu

# Time GPU version (including data transfers)
start_gpu = time.perf_counter()
d_large_data = cuda.to_device(large_data)
d_result = cuda.to_device(np.zeros_like(large_data))

threads_per_block = 256
blocks_per_grid = (large_data.size + threads_per_block - 1) // threads_per_block
gpu_square_array[blocks_per_grid, threads_per_block](d_large_data, d_result)

result_gpu = d_result.copy_to_host()
gpu_time = time.perf_counter() - start_gpu

print(f"CPU time: {cpu_time:.4f} seconds")
print(f"GPU time: {gpu_time:.4f} seconds (including data transfers)")
print(f"Speedup: {cpu_time/gpu_time:.2f}x")

# Verify results are the same
print("Results match:", np.allclose(result_cpu, result_gpu))

CPU time: 0.1302 seconds
GPU time: 0.0122 seconds (including data transfers)
Speedup: 10.70x
Results match: True


## **Pattern 2: Device Functions** <a name="device-functions"></a>

Device functions are helper functions that run on the GPU. This pattern introduces several new concepts including:
- Function composition on GPU
- Multiple GPU functions working together
- Device-specific function decoration

### Understanding Device Functions

In [6]:
# The device=True parameter indicates this is a GPU helper function
# Unlike regular CUDA kernels, device functions can return values
# They can ONLY be called from other GPU functions, not from CPU code
@cuda.jit(device=True)
def device_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# This is our main kernel that uses the device function
@cuda.jit
def gpu_calculate_distances(points_x, points_y, distances):
    idx = cuda.grid(1)
    if idx < points_x.size - 1:
        # Device functions can be called just like regular functions
        # inside GPU kernels
        distances[idx] = device_distance(
            points_x[idx], points_y[idx],
            points_x[idx + 1], points_y[idx + 1]
        )

# Example usage with detailed prints for understanding
points_x = np.array([0, 1, 2, 3], dtype=np.float64)
points_y = np.array([0, 1, 2, 3], dtype=np.float64)
distances = np.zeros(len(points_x) - 1, dtype=np.float64)

print("Input points:")
for i in range(len(points_x)):
    print(f"Point {i}: ({points_x[i]}, {points_y[i]})")

# GPU setup and execution
d_points_x = cuda.to_device(points_x)
d_points_y = cuda.to_device(points_y)
d_distances = cuda.to_device(distances)

threads_per_block = 256
blocks_per_grid = (len(points_x) + threads_per_block - 1) // threads_per_block

gpu_calculate_distances[blocks_per_grid, threads_per_block](d_points_x, d_points_y, d_distances)
distances = d_distances.copy_to_host()

print("\nCalculated distances between consecutive points:")
for i in range(len(distances)):
    print(f"Distance between points {i} and {i+1}: {distances[i]:.4f}")

Input points:
Point 0: (0.0, 0.0)
Point 1: (1.0, 1.0)
Point 2: (2.0, 2.0)
Point 3: (3.0, 3.0)

Calculated distances between consecutive points:
Distance between points 0 and 1: 1.4142
Distance between points 1 and 2: 1.4142
Distance between points 2 and 3: 1.4142


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Key New Concepts

1. **Device Functions (`@cuda.jit(device=True)`)**
   - Helper functions that run on the GPU
   - Can return values (unlike kernel functions)
   - Can only be called by other GPU functions
   - Useful for code organization and reusability
   - Don't require grid/block configuration as they're not entry points

2. **Function Composition**
   ```python
   distances[idx] = device_distance(points_x[idx], points_y[idx],
                                  points_x[idx + 1], points_y[idx + 1])
   ```
   - GPU functions can call other GPU functions
   - Helps break down complex computations
   - Improves code readability and maintainability

3. **Data Access Patterns**
   ```python
   if idx < points_x.size - 1:
   ```
   - Note the `-1` in the bounds check
   - When computing distances between consecutive points, we need N-1 distances for N points
   - Shows how to handle computations that reference adjacent elements

### Comparing Device Functions vs Kernels

| Aspect | Device Function | Kernel |
|--------|----------------|---------|
| Decoration | `@cuda.jit(device=True)` | `@cuda.jit` |
| Can Return Values | Yes | No |
| Called From | GPU functions only | CPU code |
| Grid/Block Config | Not needed | Required |
| Primary Use | Helper logic | Main entry point |

### Example: Using Multiple Device Functions

Let's extend our example to show how multiple device functions can work together:

In [7]:
@cuda.jit(device=True)
def device_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

@cuda.jit(device=True)
def device_midpoint(x1, y1, x2, y2):
    return (x1 + x2) / 2, (y1 + y2) / 2

@cuda.jit
def gpu_analyze_points(points_x, points_y, distances, midpoints_x, midpoints_y):
    idx = cuda.grid(1)
    if idx < points_x.size - 1:
        # Calculate distance using first device function
        distances[idx] = device_distance(
            points_x[idx], points_y[idx],
            points_x[idx + 1], points_y[idx + 1]
        )
        # Calculate midpoint using second device function
        midpoints_x[idx], midpoints_y[idx] = device_midpoint(
            points_x[idx], points_y[idx],
            points_x[idx + 1], points_y[idx + 1]
        )

# Example usage
points_x = np.array([0, 2, 4, 6], dtype=np.float64)
points_y = np.array([0, 2, 4, 6], dtype=np.float64)
distances = np.zeros(len(points_x) - 1, dtype=np.float64)
midpoints_x = np.zeros(len(points_x) - 1, dtype=np.float64)
midpoints_y = np.zeros(len(points_x) - 1, dtype=np.float64)

# Transfer data and execute
d_points_x = cuda.to_device(points_x)
d_points_y = cuda.to_device(points_y)
d_distances = cuda.to_device(distances)
d_midpoints_x = cuda.to_device(midpoints_x)
d_midpoints_y = cuda.to_device(midpoints_y)

threads_per_block = 256
blocks_per_grid = (len(points_x) + threads_per_block - 1) // threads_per_block

gpu_analyze_points[blocks_per_grid, threads_per_block](
    d_points_x, d_points_y, d_distances, d_midpoints_x, d_midpoints_y
)

# Get results
distances = d_distances.copy_to_host()
midpoints_x = d_midpoints_x.copy_to_host()
midpoints_y = d_midpoints_y.copy_to_host()

print("\nExtended Analysis Results:")
for i in range(len(distances)):
    print(f"Segment {i}:")
    print(f"  Distance: {distances[i]:.4f}")
    print(f"  Midpoint: ({midpoints_x[i]:.1f}, {midpoints_y[i]:.1f})")


Extended Analysis Results:
Segment 0:
  Distance: 2.8284
  Midpoint: (1.0, 1.0)
Segment 1:
  Distance: 2.8284
  Midpoint: (3.0, 3.0)
Segment 2:
  Distance: 2.8284
  Midpoint: (5.0, 5.0)


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Best Practices for Device Functions

1. Keep device functions focused on a single task
2. Use device functions to avoid code duplication in kernels
3. Consider using device functions for complex calculations that are used repeatedly
4. Remember that device functions add a small overhead compared to inlined code
5. Use them to improve code readability and maintainability when the performance impact is acceptable

## **Pattern 3: Memory Management** <a name="memory"></a>

Memory management on GPUs is crucial for performance. This pattern introduces:
- Shared memory usage
- Thread synchronization
- Memory access patterns
- Different types of GPU memory

### Understanding GPU Memory Types

1. **Global Memory**
   - Accessible by all threads across all blocks
   - Slowest type of memory
   - Used for main data storage (what we used in Patterns 1 & 2)

2. **Shared Memory**
   - Shared between threads within the same block
   - Much faster than global memory
   - Limited size (typically 48KB per block)
   - Must be explicitly managed

3. **Local Memory**
   - Private to each thread
   - Automatically managed by the compiler
   - Used for thread-local variables

### Example: Using Shared Memory for Moving Average

This example calculates a moving average of array elements using shared memory:

In [8]:
@cuda.jit
def gpu_shared_memory_example(input_array, output_array):
    # Declare shared memory array
    # Size must be known at compile time
    # Each thread block gets its own copy of shared memory
    shared_array = cuda.shared.array(256, dtype=float64)
    
    # Get thread indices
    idx = cuda.grid(1)  # Global thread ID
    thread_id = cuda.threadIdx.x  # Local thread ID within the block
    
    # Phase 1: Load data into shared memory
    if idx < input_array.size:
        shared_array[thread_id] = input_array[idx]
    else:
        # Pad with zeros if we're beyond array bounds
        shared_array[thread_id] = 0.0
    
    # CRITICAL: Synchronize threads before proceeding
    # Ensures all threads have written to shared memory
    cuda.syncthreads()
    
    # Phase 2: Process data using shared memory
    if idx < input_array.size:
        # Calculate average of current element with neighbors
        if thread_id > 0 and thread_id < 255:
            output_array[idx] = (shared_array[thread_id - 1] + 
                               shared_array[thread_id] + 
                               shared_array[thread_id + 1]) / 3
        else:
            # Handle edge cases (first and last elements)
            output_array[idx] = shared_array[thread_id]

# Create example data and show how the moving average works
data = np.array([1, 4, 6, 2, 8, 5, 9, 3], dtype=np.float64)
print("Original data:", data)

# Pad data to match shared memory size
padded_data = np.zeros(256, dtype=np.float64)
padded_data[:len(data)] = data
result = np.zeros_like(padded_data)

# Transfer data to GPU
d_data = cuda.to_device(padded_data)
d_result = cuda.to_device(result)

# Launch kernel with exactly one block of 256 threads
gpu_shared_memory_example[1, 256](d_data, d_result)

# Get results
result = d_result.copy_to_host()
print("\nMoving average results (first 8 elements):", result[:8])

Original data: [1. 4. 6. 2. 8. 5. 9. 3.]

Moving average results (first 8 elements): [1.         3.66666667 4.         5.33333333 5.         7.33333333
 5.66666667 4.        ]


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Key Concepts Explained

1. **Shared Memory Declaration**
   ```python
   shared_array = cuda.shared.array(256, dtype=float64)
   ```
   - Creates a shared memory array within each thread block
   - Size must be a compile-time constant
   - Each block gets its own independent copy
   - Faster than global memory but limited in size

2. **Thread Identification**
   ```python
   idx = cuda.grid(1)          # Global thread ID across all blocks
   thread_id = cuda.threadIdx.x  # Local thread ID within current block
   ```
   - Need both global and local IDs for different purposes
   - Global ID (idx) for accessing input/output arrays
   - Local ID (thread_id) for accessing shared memory

3. **Thread Synchronization**
   ```python
   cuda.syncthreads()
   ```
   - Forces all threads in a block to wait at this point
   - Critical when using shared memory
   - Ensures all threads have finished writing before any start reading
   - Only synchronizes threads within the same block

4. **Memory Access Pattern**
   ```python
   if thread_id > 0 and thread_id < 255:
       output_array[idx] = (shared_array[thread_id - 1] + 
                          shared_array[thread_id] + 
                          shared_array[thread_id + 1]) / 3
   ```
   - Shows how threads can access neighbors' data
   - Uses bounds checking to handle edge cases
   - Demonstrates shared memory's advantage for repeated access

### Performance Comparison

Let's compare the performance of shared memory vs global memory approaches:

In [9]:
# Global memory version (without shared memory)
@cuda.jit
def gpu_global_memory_example(input_array, output_array):
    idx = cuda.grid(1)
    if idx < input_array.size - 1 and idx > 0:
        output_array[idx] = (input_array[idx - 1] + 
                           input_array[idx] + 
                           input_array[idx + 1]) / 3

# Create larger test data
large_data = np.random.random(10000).astype(np.float64)
result_shared = np.zeros_like(large_data)
result_global = np.zeros_like(large_data)

# Test shared memory version
start_shared = time.perf_counter()
padded_data = np.zeros((((len(large_data) + 255) // 256) * 256), dtype=np.float64)
padded_data[:len(large_data)] = large_data
d_padded = cuda.to_device(padded_data)
d_result = cuda.to_device(np.zeros_like(padded_data))

blocks = (len(padded_data) + 255) // 256
gpu_shared_memory_example[blocks, 256](d_padded, d_result)
result_shared = d_result.copy_to_host()[:len(large_data)]
shared_time = time.perf_counter() - start_shared

# Test global memory version
start_global = time.perf_counter()
d_data = cuda.to_device(large_data)
d_result = cuda.to_device(result_global)

blocks = (len(large_data) + 255) // 256
gpu_global_memory_example[blocks, 256](d_data, d_result)
result_global = d_result.copy_to_host()
global_time = time.perf_counter() - start_global

print(f"Shared Memory Time: {shared_time:.6f} seconds")
print(f"Global Memory Time: {global_time:.6f} seconds")
print(f"Speedup: {global_time/shared_time:.2f}x")

Shared Memory Time: 0.001889 seconds
Global Memory Time: 0.050942 seconds
Speedup: 26.96x


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Best Practices for GPU Memory Management

1. **Shared Memory Usage**
   - Use shared memory when data will be accessed multiple times
   - Be mindful of shared memory size limitations
   - Consider padding to avoid bank conflicts

2. **Thread Synchronization**
   - Always synchronize after shared memory writes
   - Minimize synchronization points
   - Remember synchronization only works within a block

3. **Memory Access Patterns**
   - Strive for Aligned memory access
   - Minimize divergent branching
   - Consider padding to achieve better memory alignment

4. **Data Transfer**
   - Minimize host-device transfers
   - Batch data transfers when possible
   - Consider using pinned memory for faster transfers

5. **Memory Allocation**
   - Reuse allocated memory when possible
   - Free memory when no longer needed
   - Consider using memory pools for dynamic allocation

## **Pattern 4: Kernel Launch Patterns** <a name="kernels"></a>

Understanding how to structure and launch kernels for multi-dimensional problems is crucial for GPU programming. This pattern introduces:
- 2D grid and block configurations
- Multi-dimensional thread indexing
- Matrix operations on GPU
- Advanced launch configurations

### Matrix Transposition Example

We'll implement matrix transposition as it's a perfect example for demonstrating 2D grid operations:

In [12]:
@cuda.jit
def gpu_matrix_transpose(input_matrix, output_matrix):
    # Get 2D thread indices
    row, col = cuda.grid(2)
    
    # Bounds checking is crucial for 2D operations
    if row < input_matrix.shape[0] and col < input_matrix.shape[1]:
        # Transpose operation: swap row and column indices
        output_matrix[col, row] = input_matrix[row, col]

# Create a sample matrix
matrix = np.array([
    [1, 2, 3],
    [4, 5, 6]
], dtype=np.float64)

print("Original matrix:")
print(matrix)
print("\nMatrix shape:", matrix.shape)

# Create output matrix with transposed shape
result = np.zeros((matrix.shape[1], matrix.shape[0]), dtype=np.float64)

# 2D grid and block configuration
threads_per_block_2d = (16, 16)  # 16x16 = 256 threads per block
blocks_per_grid_2d = (
    (matrix.shape[0] + threads_per_block_2d[0] - 1) // threads_per_block_2d[0],
    (matrix.shape[1] + threads_per_block_2d[1] - 1) // threads_per_block_2d[1]
)

print("\nGrid Configuration:")
print(f"Threads per block: {threads_per_block_2d}")
print(f"Blocks per grid: {blocks_per_grid_2d}")

# Transfer data and execute
d_matrix = cuda.to_device(matrix)
d_result = cuda.to_device(result)

gpu_matrix_transpose[blocks_per_grid_2d, threads_per_block_2d](d_matrix, d_result)

result = d_result.copy_to_host()
print("\nTransposed matrix:")
print(result)

Original matrix:
[[1. 2. 3.]
 [4. 5. 6.]]

Matrix shape: (2, 3)

Grid Configuration:
Threads per block: (16, 16)
Blocks per grid: (1, 1)

Transposed matrix:
[[1. 4.]
 [2. 5.]
 [3. 6.]]


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Key New Concepts

1. **2D Grid Configuration**
   ```python
   threads_per_block_2d = (16, 16)
   blocks_per_grid_2d = (
       (matrix.shape[0] + threads_per_block_2d[0] - 1) // threads_per_block_2d[0],
       (matrix.shape[1] + threads_per_block_2d[1] - 1) // threads_per_block_2d[1]
   )
   ```
   - Two-dimensional thread blocks (16×16 = 256 threads per block)
   - Grid dimensions calculated for each dimension separately
   - Ceiling division ensures coverage of non-perfectly-divisible dimensions

2. **2D Thread Indexing**
   ```python
   row, col = cuda.grid(2)
   ```
   - cuda.grid(2) returns a 2D tuple of indices
   - Automatically calculates global position from block and thread indices
   - Equivalent to manual calculation:
     ```python
     row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
     col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
     ```

3. **2D Bounds Checking**
   ```python
   if row < input_matrix.shape[0] and col < input_matrix.shape[1]:
   ```
   - Must check bounds for both dimensions
   - Prevents out-of-bounds access in non-square matrices
   - Handles cases where grid size exceeds matrix dimensions


### Extended Example: Matrix Operations with Different Launch Patterns

Let's explore three different ways to structure matrix transposition, each with its own advantages and trade-offs:
1. 2D Grid (Original approach)
2. Row-major approach (1D grid, each thread handles a row)
3. Column-major approach (1D grid, each thread handles a column)

In [15]:
@cuda.jit
def gpu_transpose_row_major(input_matrix, output_matrix):
    # Single thread processes entire row
    # blockIdx.x gives the block index
    # blockDim.x gives the number of threads per block
    # threadIdx.x gives the thread index within the block
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    
    if row < input_matrix.shape[0]:
        # One thread processes entire row
        # Advantage: Good memory coalescing for input matrix
        # Disadvantage: Poor memory coalescing for output matrix
        for col in range(input_matrix.shape[1]):
            output_matrix[col, row] = input_matrix[row, col]

# Version 2: Column-major processing
@cuda.jit
def gpu_transpose_col_major(input_matrix, output_matrix):
    # Single thread processes entire column
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    
    if col < input_matrix.shape[1]:
        # One thread processes entire column
        # Advantage: Good memory coalescing for output matrix
        # Disadvantage: Poor memory coalescing for input matrix
        for row in range(input_matrix.shape[0]):
            output_matrix[col, row] = input_matrix[row, col]

# Version 3: 2D grid with shared memory (more advanced approach)
@cuda.jit
def gpu_transpose_shared_memory(input_matrix, output_matrix):
    # Declare shared memory - size must be known at compile time
    TILE_DIM = 16  # Should match block size for efficiency
    tile = cuda.shared.array(shape=(TILE_DIM, TILE_DIM), dtype=float64)
    
    # Get 2D thread indices
    row, col = cuda.grid(2)
    block_row, block_col = cuda.threadIdx.x, cuda.threadIdx.y
    
    if row < input_matrix.shape[0] and col < input_matrix.shape[1]:
        # Load data into shared memory
        # This read is Aligned
        tile[block_row, block_col] = input_matrix[row, col]
        
        # Wait for all threads to load their data
        cuda.syncthreads()
        
        # Calculate output indices
        out_row = col
        out_col = row
        
        # Write to output matrix with transposed indices
        # Shared memory helps make this write more efficient
        if out_row < output_matrix.shape[0] and out_col < output_matrix.shape[1]:
            output_matrix[out_row, out_col] = tile[block_row, block_col]

# Create a larger test matrix for meaningful comparison
test_matrix = np.arange(32, dtype=np.float64).reshape(8, 4)
print("Original matrix:")
print(test_matrix)
print("\nShape:", test_matrix.shape)

# Prepare output arrays for each version
result_2d = np.zeros((test_matrix.shape[1], test_matrix.shape[0]), dtype=np.float64)
result_row = np.zeros_like(result_2d)
result_col = np.zeros_like(result_2d)
result_shared = np.zeros_like(result_2d)

# Transfer input data to GPU
d_test = cuda.to_device(test_matrix)

# 1. Test 2D version (original)
print("\n1. Testing 2D Grid Version:")
threads_2d = (16, 16)
blocks_2d = (
    (test_matrix.shape[0] + threads_2d[0] - 1) // threads_2d[0],
    (test_matrix.shape[1] + threads_2d[1] - 1) // threads_2d[1]
)
print(f"Grid config: {blocks_2d} blocks, {threads_2d} threads per block")

d_result = cuda.to_device(result_2d)
start = time.perf_counter()
gpu_matrix_transpose[blocks_2d, threads_2d](d_test, d_result)
cuda.synchronize()
time_2d = time.perf_counter() - start
result_2d = d_result.copy_to_host()

# 2. Test Row-major version
print("\n2. Testing Row-Major Version:")
threads_1d = 256
blocks_row = (test_matrix.shape[0] + threads_1d - 1) // threads_1d
print(f"Grid config: {blocks_row} blocks, {threads_1d} threads per block")

d_result = cuda.to_device(result_row)
start = time.perf_counter()
gpu_transpose_row_major[blocks_row, threads_1d](d_test, d_result)
cuda.synchronize()
time_row = time.perf_counter() - start
result_row = d_result.copy_to_host()

# 3. Test Column-major version
print("\n3. Testing Column-Major Version:")
blocks_col = (test_matrix.shape[1] + threads_1d - 1) // threads_1d
print(f"Grid config: {blocks_col} blocks, {threads_1d} threads per block")

d_result = cuda.to_device(result_col)
start = time.perf_counter()
gpu_transpose_col_major[blocks_col, threads_1d](d_test, d_result)
cuda.synchronize()
time_col = time.perf_counter() - start
result_col = d_result.copy_to_host()

# 4. Test Shared Memory version
print("\n4. Testing Shared Memory Version:")
d_result = cuda.to_device(result_shared)
start = time.perf_counter()
gpu_transpose_shared_memory[blocks_2d, threads_2d](d_test, d_result)
cuda.synchronize()
time_shared = time.perf_counter() - start
result_shared = d_result.copy_to_host()

# Compare results and timing
print("\nPerformance Comparison:")
print(f"2D Grid Version: {time_2d*1000:.3f} ms")
print(f"Row-Major Version: {time_row*1000:.3f} ms")
print(f"Column-Major Version: {time_col*1000:.3f} ms")
print(f"Shared Memory Version: {time_shared*1000:.3f} ms")

print("\nAll results match:", 
      np.allclose(result_2d, result_row) and 
      np.allclose(result_2d, result_col) and
      np.allclose(result_2d, result_shared))

Original matrix:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]
 [16. 17. 18. 19.]
 [20. 21. 22. 23.]
 [24. 25. 26. 27.]
 [28. 29. 30. 31.]]

Shape: (8, 4)

1. Testing 2D Grid Version:
Grid config: (1, 1) blocks, (16, 16) threads per block

2. Testing Row-Major Version:
Grid config: 1 blocks, 256 threads per block

3. Testing Column-Major Version:
Grid config: 1 blocks, 256 threads per block

4. Testing Shared Memory Version:


c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\caleb\anaconda3\envs\collisionEnv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))



Performance Comparison:
2D Grid Version: 7.637 ms
Row-Major Version: 57.077 ms
Column-Major Version: 56.860 ms
Shared Memory Version: 192.973 ms

All results match: True


### Understanding the Different Approaches

1. **2D Grid Version**
   - Uses a 2D grid of threads
   - Each thread handles one element
   - Advantages:
     - Simple to understand
     - Good parallelism
   - Disadvantages:
     - Memory access pattern not optimal
     - More thread management overhead

2. **Row-Major Version**
   - Uses a 1D grid of threads
   - Each thread handles an entire row
   - Advantages:
     - Aligned memory access for input matrix
     - Simpler thread management
   - Disadvantages:
     - Less parallelism
     - Poor memory access pattern for output matrix

3. **Column-Major Version**
   - Uses a 1D grid of threads
   - Each thread handles an entire column
   - Advantages:
     - Aligned memory access for output matrix
     - Simpler thread management
   - Disadvantages:
     - Less parallelism
     - Poor memory access pattern for input matrix

4. **Shared Memory Version**
   - Uses 2D grid with shared memory tile
   - Each thread block cooperatively loads and stores data
   - Advantages:
     - Better memory access patterns
     - Can be much faster for larger matrices
   - Disadvantages:
     - More complex
     - Limited by shared memory size
     - Requires careful synchronization

### Key Takeaways for Launch Patterns

1. **Memory Access Patterns Matter**
   - Coalesced memory access is crucial for performance
   - Different approaches have different memory access characteristics
   - Consider both read and write patterns

2. **Thread Organization**
   - 1D vs 2D grids affect parallelism and complexity
   - Consider the problem structure when choosing
   - Balance parallelism with overhead

3. **Shared Memory Usage**
   - Can dramatically improve performance
   - Requires careful synchronization
   - Limited resource - use wisely

4. **Workload Distribution**
   - Consider how work is divided among threads
   - Balance thread utilization
   - Watch for divergent execution paths

## Best Practices and Common Pitfalls <a name="best-practices"></a>

### 1. Memory Transfer Minimization
- Minimize data transfers between CPU and GPU
- Use asynchronous transfers when possible
- Batch operations to reduce transfer overhead

### 2. Thread Divergence
- Avoid conditional statements that cause threads in a warp to take different paths
- Structure conditions to affect entire warps when possible

### 3. Memory Coalescing
- Access memory in contiguous chunks
- Align data structures to memory boundaries

### 4. Occupancy
- Balance thread block size with resource usage
- Consider shared memory usage impact on occupancy

### 5. Common Pitfalls
- Not checking array bounds
- Incorrect grid/block size calculations
- Not synchronizing threads when necessary
- Using too much shared memory

## Performance Tips

1. Profile your code using NVIDIA's profiling tools
2. Experiment with different block sizes
3. Use appropriate data types (float32 vs float64)
4. Consider memory access patterns
5. Balance parallelism with resource usage

## Practice Exercises

1. Convert a vector dot product function to GPU code
2. Implement a matrix multiplication kernel
3. Create a parallel histogram calculation
4. Implement a parallel prefix sum (scan) operation

In [ ]:
import numpy as np
import time
from numba import cuda, float32
import math

"""
CUDA Conversion Practice Exercises
--------------------------------
Your task is to implement the body of each GPU kernel and its corresponding launch function.
The CPU implementation and verification code is provided for each exercise.
"""

# Exercise 1: Vector Dot Product  (Level: Beginner)
# Learning objectives:
# - Basic CUDA kernel structure
# - Simple parallel reduction
# - Using shared memory for partial results
# - Atomic operations for final sum

def cpu_dot_product(a, b):
    """
    Compute the dot product of two vectors.
    
    Parameters:
        a, b: numpy arrays of same length
    Returns:
        float: dot product result
    """
    return np.sum(a * b)

@cuda.jit
def gpu_dot_product_kernel(a, b, result):
    """
    CUDA kernel for vector dot product.
    
    Parameters:
        a, b: device arrays of same length
        result: device array of length 1 to store final dot product
    """
    # TODO: Implement the kernel
    # Hints:
    # 1. Use shared memory for partial sums within each block
    # 2. Each thread should process one or more pairs of elements
    # 3. Use atomic add for the final reduction
    pass

def gpu_dot_product(a, b):
    """Launch function for dot product kernel"""
    # TODO: Implement kernel launch configuration and data transfer
    pass

def verify_dot_product(n=1000000):
    """Verification function for your GPU implementation"""
    # Create test data
    a = np.random.random(n).astype(np.float32)
    b = np.random.random(n).astype(np.float32)
    
    # Get CPU result for comparison
    cpu_start = time.time()
    cpu_result = cpu_dot_product(a, b)
    cpu_time = time.time() - cpu_start
    
    # Get GPU result
    gpu_start = time.time()
    gpu_result = gpu_dot_product(a, b)
    gpu_time = time.time() - gpu_start
    
    print(f"Vector Dot Product Results (n={n:,}):")
    print(f"CPU Result: {cpu_result:.6f}, Time: {cpu_time:.6f} seconds")
    print(f"GPU Result: {gpu_result:.6f}, Time: {gpu_time:.6f} seconds")
    if gpu_time > 0:
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    print(f"Absolute difference: {abs(cpu_result - gpu_result):.10f}\n")

# Exercise 2: Matrix Multiplication (Level: Intermediate)
# Learning objectives:
# - 2D grid and block structure
# - Tiled matrix multiplication
# - Shared memory for data reuse
# - Memory coalescing considerations

def cpu_matrix_multiply(a, b):
    """
    Compute the product of two matrices.
    
    Parameters:
        a: numpy 2D array of shape (M, K)
        b: numpy 2D array of shape (K, N)
    Returns:
        numpy 2D array of shape (M, N): result of a @ b
    """
    return np.dot(a, b)

@cuda.jit
def gpu_matrix_multiply_kernel(a, b, c):
    """
    CUDA kernel for matrix multiplication.
    
    Parameters:
        a: device array of shape (M, K)
        b: device array of shape (K, N)
        c: device array of shape (M, N) for result
    """
    # TODO: Implement the kernel
    # Hints:
    # 1. Use shared memory for tiles of input matrices
    # 2. Each thread should compute one element of output
    # 3. Ensure coalesced memory access
    pass

def gpu_matrix_multiply(a, b):
    """Launch function for matrix multiplication kernel"""
    # TODO: Implement kernel launch configuration and data transfer
    pass

def verify_matrix_multiply(n=1000):
    """Verification function for your GPU implementation"""
    # Create test matrices
    a = np.random.random((n, n)).astype(np.float32)
    b = np.random.random((n, n)).astype(np.float32)
    
    # Get CPU result
    cpu_start = time.time()
    cpu_result = cpu_matrix_multiply(a, b)
    cpu_time = time.time() - cpu_start
    
    # Get GPU result
    gpu_start = time.time()
    gpu_result = gpu_matrix_multiply(a, b)
    gpu_time = time.time() - gpu_start
    
    print(f"Matrix Multiplication Results (n={n}x{n}):")
    print(f"CPU Time: {cpu_time:.6f} seconds")
    print(f"GPU Time: {gpu_time:.6f} seconds")
    if gpu_time > 0:
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    print(f"Max absolute difference: {np.max(np.abs(cpu_result - gpu_result)):.10f}\n")

# Exercise 3: Histogram Calculation (Level: Intermediate-Advanced)
# Learning objectives:
# - Atomic operations for concurrent updates
# - Handling race conditions
# - Memory access patterns
# - Load balancing

def cpu_histogram(data, nbins, min_val, max_val):
    """
    Compute histogram of input data.
    
    Parameters:
        data: numpy array of values
        nbins: number of histogram bins
        min_val: minimum value for binning
        max_val: maximum value for binning
    Returns:
        numpy array: histogram counts
    """
    return np.histogram(data, bins=nbins, range=(min_val, max_val))[0]

@cuda.jit
def gpu_histogram_kernel(data, histogram, min_val, max_val):
    """
    CUDA kernel for histogram calculation.
    
    Parameters:
        data: input device array
        histogram: device array for histogram bins
        min_val: minimum value for binning
        max_val: maximum value for binning
    """
    # TODO: Implement the kernel
    # Hints:
    # 1. Calculate bin index for each element
    # 2. Use atomic operations to update bins
    # 3. Handle out-of-range values
    pass

def gpu_histogram(data, nbins, min_val, max_val):
    """Launch function for histogram kernel"""
    # TODO: Implement kernel launch configuration and data transfer
    pass

def verify_histogram(n=10000000, nbins=128):
    """Verification function for your GPU implementation"""
    # Create test data
    data = np.random.random(n).astype(np.float32)
    min_val, max_val = 0, 1
    
    # Get CPU result
    cpu_start = time.time()
    cpu_result = cpu_histogram(data, nbins, min_val, max_val)
    cpu_time = time.time() - cpu_start
    
    # Get GPU result
    gpu_start = time.time()
    gpu_result = gpu_histogram(data, nbins, min_val, max_val)
    gpu_time = time.time() - gpu_start
    
    print(f"Histogram Calculation Results (n={n:,}, bins={nbins}):")
    print(f"CPU Time: {cpu_time:.6f} seconds")
    print(f"GPU Time: {gpu_time:.6f} seconds")
    if gpu_time > 0:
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    print(f"Max absolute difference: {np.max(np.abs(cpu_result - gpu_result))}\n")

# Exercise 4: Parallel Prefix Sum (Scan) (Level: Advanced)
# Learning objectives:
# - Complex shared memory patterns
# - Multi-phase algorithms
# - Block synchronization
# - Work-efficient parallel algorithms

def cpu_prefix_sum(data):
    """
    Compute inclusive prefix sum (cumulative sum) of input array.
    
    Parameters:
        data: numpy array
    Returns:
        numpy array: cumulative sum
    """
    return np.cumsum(data)

@cuda.jit
def gpu_prefix_sum_kernel(data, output):
    """
    CUDA kernel for prefix sum calculation.
    
    Parameters:
        data: input device array
        output: device array for results
    """
    # TODO: Implement the kernel
    # Hints:
    # 1. Implement Blelloch scan algorithm
    # 2. Use shared memory for block-level scan
    # 3. Handle boundary conditions
    pass

def gpu_prefix_sum(data):
    """Launch function for prefix sum kernel"""
    # TODO: Implement kernel launch configuration and data transfer
    pass

def verify_prefix_sum(n=1000000):
    """Verification function for your GPU implementation"""
    # Create test data
    data = np.random.random(n).astype(np.float32)
    
    # Get CPU result
    cpu_start = time.time()
    cpu_result = cpu_prefix_sum(data)
    cpu_time = time.time() - cpu_start
    
    # Get GPU result
    gpu_start = time.time()
    gpu_result = gpu_prefix_sum(data)
    gpu_time = time.time() - gpu_start
    
    print(f"Prefix Sum Results (n={n:,}):")
    print(f"CPU Time: {cpu_time:.6f} seconds")
    print(f"GPU Time: {gpu_time:.6f} seconds")
    if gpu_time > 0:
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    print(f"Max absolute difference: {np.max(np.abs(cpu_result - gpu_result)):.10f}\n")

if __name__ == "__main__":
    print("\nExercise 1: Vector Dot Product")
    print("------------------------------")
    verify_dot_product()
    
    print("\nExercise 2: Matrix Multiplication")
    print("--------------------------------")
    verify_matrix_multiply()
    
    print("\nExercise 3: Histogram Calculation")
    print("--------------------------------")
    verify_histogram()
    
    print("\nExercise 4: Parallel Prefix Sum")
    print("------------------------------")
    verify_prefix_sum()

# 🎉 Congratulations!

You've completed the CUDA programming exercises! By working through these examples, you've gained hands-on experience with fundamental GPU programming concepts:

## 🎯 Key Concepts Covered
- Basic CUDA kernel structure and memory management  
- Parallel reduction techniques
- Shared memory optimization 
- Memory coalescing and access patterns
- Atomic operations
- Block and grid dimensioning
- Complex parallel algorithms

## 📚 Next Steps
To continue your GPU programming journey, consider:
1. Experimenting with different block and grid sizes to optimize performance
2. Adding error handling to your implementations 
3. Trying the same algorithms with larger datasets
4. Implementing other parallel algorithms like:
  - Sorting algorithms
  - Graph algorithms
  - Image processing kernels
  - Stencil computations

## 🔧 Performance Tips
Remember these key optimization strategies:
- Minimize memory transfers between CPU and GPU
- Use shared memory when possible
- Ensure coalesced memory access
- Balance occupancy and resource usage 
- Profile your code to identify bottlenecks

## 📖 Useful Resources
- [CUDA Documentation](https://docs.nvidia.com/cuda/)
- [Numba Documentation](https://numba.pydata.org/)
- [CUDA C++ Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)
- [CUDA Blog](https://developer.nvidia.com/blog/tag/cuda/)

## 💡 Pro Tips
- Always profile before optimizing
- Start with simple implementations and optimize incrementally
- Use power-of-2 sizes for arrays when possible
- Consider memory alignment and access patterns
- Test with various input sizes

## 🐞 Debugging Tips
- Use `cuda.syncthreads()` judiciously
- Print intermediate results for small inputs
- Check array bounds carefully
- Verify results against CPU implementation
- Use CUDA-GDB for complex issues

Thanks for completing this tutorial! If you have questions or feedback, please fill out this [form](https://forms.gle/CZYBL9zVfPTixHFW6)